In [7]:
# 여러 대 미사일 발사 코드 구상 : 시간 제한으로 구현 pygame.time 사용
import pygame
import random
import time

# 1. 초기화 -> 변수의 선언
pygame.init()

# 2. 게임 화면 설정 -> 고정값
size = [600, 800]
screen = pygame.display.set_mode(size)
title = 'pyamge example'
pygame.display.set_caption(title)

# 3. 게임 내에서의 설정 -> 변수
clock = pygame.time.Clock()
color = (83, 104, 154)

class Img:
    def __init__(self):
        self.width = 0
        self.height = 0
        self.x = 0
        self.y = 0
        self.speed = 0

    def img_load_size(self, address, width, height):
        self.img = pygame.image.load(address)
        self.img = pygame.transform.scale(self.img, (width, height))
        self.width = width
        self.height = height

    def show_img(self):
        screen.blit(self.img, (self.x, self.y))

    def set_speed(self, speed):
        self.speed = speed

def is_crash(a, b):
    if a.x < b.x + b.width and a.x + a.width > b.x :
        if a.y + a.height > b.y and a.y < b.y + b.height:
            return True
    return False

hero = Img()
hero.img_load_size('C:/Users/admin/Desktop/pygame/spaceship.png', 100, 100)
hero.x = (size[0]/2) - (hero.width/2)
hero.y = size[1] - hero.height - 30
hero.set_speed(7)


score = 0
game_over = 0
missile_list = []
enemy_list = []
enemy_delete = []
font = pygame.font.Font('C:/Windows/Fonts/Arial.ttf', 22)

right_move = False
left_move = False
up_move = False
down_move = False
spacebar = False

# missile_speed = 0
last_shot_time = 0
shoot_interval = 250 # 발사간격 (밀리초 단위)

# 4. 메인 이벤트
system = 0
while system == 0:
# 4-1. FPS(Frame per sec) 설정
    clock.tick(60)

    current_time = pygame.time.get_ticks() # 현재 시간 구하는 코드

# 4-2. 입력(키보드,마우스)의 감지 -> 활용
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            system = 1

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                left_move = True
            if event.key == pygame.K_RIGHT:
                right_move = True
            if event.key == pygame.K_UP:
                up_move = True
            if event.key == pygame.K_DOWN:
                down_move = True
            if event.key == pygame.K_SPACE:
                spacebar = True

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                left_move = False
            if event.key == pygame.K_RIGHT:
                right_move = False
            if event.key == pygame.K_UP:
                up_move = False
            if event.key == pygame.K_DOWN:
                down_move = False
            if event.key == pygame.K_SPACE:
                spacebar = False

# 4-3. 입력에 따른 변화, 시간에 따른 변화
    if right_move == True :
        hero.x += hero.speed
    if left_move == True :
        hero.x -= hero.speed
    if up_move == True :
        hero.y -= hero.speed
    if down_move == True :
        hero.y += hero.speed

    if hero.x < 0:
        hero.x = 0
    if hero.x > (size[0] - hero.width):
        hero.x = size[0] - hero.width
    if hero.y < 0:
        hero.y = 0
    if hero.y > size[1] - hero.height:
        hero.y = size[1] - hero.height

    if spacebar :
        # if missile_speed % 7 == 0:
        if current_time - last_shot_time > shoot_interval:
            missile = Img()
            missile.img_load_size('C:/Users/admin/Desktop/pygame/missile.png', 50,50)
            missile.x = (hero.x + hero.width/2) - (missile.width/2)
            missile.y = hero.y - missile.height - 10
            missile_list.append(missile)
            
        # missile_speed += 1
            last_shot_time = current_time    

    for missile in missile_list:
        missile.y -= 10
        if missile.y <= 0:
            missile_list.remove(missile)
    if random.random() > 0.99:
        enemy = Img()
        enemy.img_load_size('C:/Users/admin/Desktop/pygame/images.png', 50, 50)
        enemy.x = random.randint(0,size[0]-80)
        enemy.y = 5        
        enemy_list.append(enemy)


    enemy_delete = []
    for enemy in enemy_list:
        enemy.y += 5
        if enemy.y > size[1]:
            enemy_delete.append(enemy)
        
    for e in enemy_delete:
        enemy_list.remove(e)

    #충돌 처리

    for enemy in enemy_list:
        if is_crash(enemy, hero):
            game_over = 1
            system = 1
            
    for enemy in enemy_list:
        for missile in missile_list:
            if is_crash(enemy, missile):
                enemy_list.remove(enemy)
                missile_list.remove(missile)
                score += 1
    # hero_rect = pygame.Rect(hero.x, hero.y, hero.width, hero.height)
    # enemy_rect = pygame.Rect(enemy.x, enemy.y, enemy.width, enemy.height)
    # missile_rect = pygame.Rect(missile.x, missile.y , missile.width, missile.height)
    
    # if hero_rect.colliderect(enemy_rect):
    #     system = 1

    # missile_delete = []
    # for i in range(len(missile_list)):
    #     missile_list[i].y -= 10
        
    #     if missile_list[i].y < 0:
    #         missile_delete.append(missile_list[i]) # 추가 안됨 문제!(전역 변수 문제 엿음)
                        
    # for d in missile_delete:
    #     missile_list.remove(d)
    # missile_delete.clear()
        

# 4-4. 전사작업(그리기)
    
    screen.fill(color)
    hero.show_img()
    for enemy in enemy_list:
        enemy.show_img()

    
    for missile in missile_list:
        missile.show_img()

    text = font.render('SCORE : {}'.format(score), True, 'yellow')
    screen.blit(text, (10,10))



# 4-5. 업데이트
    pygame.display.flip()



# 5. 종료(x버튼, game over)
if game_over == 1 or system == 1:
    text1 = font.render('game over', True, 'red')
    screen.blit(text1, (size[0]/2 - 50, size[1]/2))
    pygame.display.flip()
    time.sleep(1)
    pygame.quit()